## Plotting and Curve_fit code for 3 PGMI Experiements

In [1]:
##Import Packages, not all of these will be needed##
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
from numpy import loadtxt
from array import array
import sys
from scipy.signal import find_peaks
import imageio.v2 as imageio

np.set_printoptions(threshold=sys.maxsize)

### Helper Functions

In [2]:

#define the interference fringe pattern fitting function
def cosine_function(x, a, b, phi, p):
    return a * np.cos(p*x - phi) + b

#prints the full function so it can be displayed as a label on our plots
def print_function(popt):
    a = round(popt[0], 2)
    b = round(popt[1], 2)
    phi = round(popt[2], 2)
    p = round(popt[3], 2)
    if phi > 0:
        phi = " +" + str(phi)
    complete_function = str(a) + "cos(" + str(p) + "x " + str(phi) + ")" + " + " + str(b)
    text = "The function is: " + complete_function
    return text

#takes in an image in textfile form and returns a numpy array of pixel values for any row in that image for fitting
def textfile_to_array(textfile_name, row):
   raw_array = np.loadtxt(textfile_name, converters= float, dtype = "int")
   raw_array = raw_array[row]
   return raw_array

#takes in image in textfile form and returns it as a numpy array, helpful for the laser power vs pixel value fit portion 
def textfile_to_array_no_row(textfile_name):
    raw_array = np.loadtxt(textfile_name)
    return raw_array

#This creates image names, especially useful for phase stepping. 
def generate_file_names(number_of_images, interval, starting_value): 
    index = np.array(range(1, number_of_images + 1))
    name_list = [] #for CameraMotorControl & LaserMotorControl use
    text_file_name_list = [] #for plots and fitting 
    value = starting_value - interval

    for i in index:
        value = value + interval
        name_list.append(str(value)) #can add more descriptors like "g1" here
        text_file_name_list.append(str(value) + ".txt")
    return name_list, text_file_name_list

### 1. Take in Phase Stepping Images & Plots Row of Pixels for Each One. Measure & Label Pixel Darkness Value of the Same Specified Pixel in Each Plot. 

This section of code simply takes in your images as arrays of text files, plots a row of data from each, marks & labels the same pixel in each and then saves each plot as an image file. It also outputs an array of pixel darkness values in each image for the pixel you are tracking to prepare for fitting. No fitting is done here. 

#### You will need to input the list of text file names, the index of the pixel(s) you wish to track/target, the index of the row that you wish to plot in each image, file names and plot labels. 

In [ ]:
##the code is currently set up to track 2 different pixels, this can be modified to track more or less pixels. 


### Copy your list of text file names here. You could also call generate_file_names() instead ###
filename_list = ["0g1.txt", "6g1.txt", "12g1.txt", "18g1.txt"]

### Change your Image Row Number. 340 is the centre of the image (better image quality) and what I typically used ###
row_number = 340

### Change your Tracking Pixel Index(s) Here ###
###To check that you selected the right pixel(s) (it will be marked with a coloured triangle in the plot), you can uncomment line 16 to just generate and view one plot. 
pixel_1_index = 474
pixel_2_index = 1011
#filename_list = ["0g1.txt"]

###Change your Plot Labels if Desired ###
xlabel = "Pixel Number"
ylabel = "Pixel Darkness"

##create our pixel darkness value arrays
target_pixel_1 = []
target_pixel_2 = []

##generates a plot for each image 
for textfile_name in filename_list:  
    ##takes arrays from text files and retrieves pixel darkness values
    ydata = textfile_to_array(textfile_name, row_number)
    xdata = np.array(range(1, 1281))
    target_pixel_1.append(ydata[pixel_1_index])
    target_pixel_2.append(ydata[pixel_2_index])
    
    ##prints pixel darkness values for convenience
    print("pixel value 1 = " + str(ydata[pixel_1_index]))
    print("pixel value 2 = " + str(ydata[pixel_2_index]))

    ##generates titles for each plot. The title is the same as the image name
    plot_name = textfile_name.split(".")
    plot_name = plot_name[0]

    ##plot all pixels, label and plot the target pixels with different colours, label plot
    plt.plot(xdata, ydata, "b.", label="data points")
    plt.plot(xdata[pixel_1_index], ydata[pixel_1_index], "c^", label="pixel 1")
    plt.plot(xdata[pixel_2_index], ydata[pixel_2_index], "m^", label="pixel 2")
    plt.title(plot_name)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    ##label the pixel values on plot
    target_pixel_1_label = "1:" + str(target_pixel_1)
    plt.text(1295, 200, target_pixel_1_label) ##Adjust label placement by changing first 2 parameters (x & y coordinates)
    target_pixel_2_label = "2: " + str(target_pixel_2)
    plt.text(1295, 250, target_pixel_2_label)

    ##Save and show plot 
    plt.legend()
    plt.savefig(plot_name + "samepixel.png") ### Change Filename Here ###
    plt.show()
    plt.clf()

    ##Append pixel darkness value for each plot into pixel darkness array. 
    target_pixel_1_array = np.array(target_pixel_1)
    target_pixel_2_array = np.array(target_pixel_2)

### 1.5. Create GIF of Plots to Visualize Change in Pixel Darkness Value

This step is optional but creating a GIF can be helpful as a sanity check, for a presentation, etc. This does not create a GIF that loops infinitely so it's best to convert it to one (there are sources online) if you are using it for a presentation. 

#### You will need to input the filenames of your plots (which would be image files) and the GIF filename. 

In [ ]:
### CHANGE YOUR PLOT FILE NAMES HERE ### 
###You can call generate_file_names() alternatively 
plotfilenames = ["0g1.png", "6g1.png", "12g1.png", "18g1.png"]
images = [ ]
gif_name = "pixels.gif"

for name in plotfilenames:
  images.append(imageio.imread(name))

###Save GIF
imageio.mimsave(gif_name, images, duration = 1000)


### 2. Make Plots of just the Pixel Darkness Values for the Target Pixels. Fit a Sinusoidal Function to these Plots & Display the Function

This section of code takes in the pixel darkness arrays for our selected pixels, makes a plot for each of them and then fits each one to the cosine function described in the helper functions section above. Assuming the data has no issues, the contrast can be calculated right after this step. 

#### You will need to input your X-axis for each plot (I suggest using the phase stepping distances as I have done here), initial cosine parameter guesses for the fit, filenames and plot labels

In [ ]:
##this code section will generate two plots, duplicate the plot 2 code & labels section if you increase the number of pixels

### FOR BOTH PLOTS ###

### CHANGE YOUR PLOTS' X-AXIS HERE ###
###Input an integer list of x-axis values that correspond to your first pixel's darkness values. 
###This will likely be the same for each tracked pixel but do add a different xdata variable if the axes are different for different pixels. 
xdata = [0, 6, 12, 18]

### CHANGE YOUR PLOTS' TITLE & AXES LABELS HERE ###
#For Plot 1
title_1 = "Pixel 1"
xlabel_1 = "Grating Positions in Micrometers"
ylabel_1 = "Pixel Darkness"
plotfilename_1 = "Pixel1Fit.png"

#For Plot 2
title_2 = "Pixel 2"
xlabel_2 = "Grating Positions in Micrometers"
ylabel_2 = "Pixel Darkness"
plotfilename_2 = "Pixel2Fit.png"

##Creates a numpy array the same size as xdata to help with indexing
xdataindex = np.arange(1, len(xdata)+1)



####PLOT 1####
##Define our first plot's Y-axis
ydata1 = target_pixel_1_array 

### CHANGE YOUR FUNCTION PARAMETER GUESSES HERE ###
###Generate plot & determine parameter estimates by inspection from plot
initialguess1 = [300, 500, 50, np.pi * 2 / 200]  # [a, b, phi, np.pi * 2 / period]

##Plot the Tracked/target Pixel Data & Add Plot & Data Labels 
plt.plot(xdata, ydata1, "c^", label="pixel 1")
plt.title(title_1)
plt.xlabel(xlabel_1)
plt.ylabel(ylabel_1)
#add data labels (can remove if this clutters the plot)
for value in xdataindex:
    plt.text(xdata[value - 1], ydata1[value - 1], str(ydata1[value - 1]))

##Generate the fitted function
popt, pcov = curve_fit(cosine_function, xdata, ydata1, initialguess1)

##plot the fitted function
fittedxvalues = np.arange(1, xdata[-1], 0.1)  ##make the x values for our fitted function. Can change parameters for more precision
fittedyvalues = cosine_function(fittedxvalues, *popt)  ##astrix allows us to pass both elements in popt into the function

##Display our function on our plot 
function_label1 = print_function(popt)
plt.text(0, max(ydata1) + 20, function_label1) ##change the x & y coordinates of the function's location as needed

##Save and show plot
plt.plot(fittedxvalues, fittedyvalues, "b", label="fitted function")
plt.legend()
plt.savefig(plotfilename_1)
plt.show()
plt.clf()


#### PLOT 2 ####

##Define our second plot's Y-axis
ydata2 = target_pixel_2_array 

### CHANGE YOUR FUNCTION PARAMETER GUESSES HERE ###
###Generate plot & determine parameter estimates by inspection from plot
initialguess2 = [300, 500, 50, np.pi * 2 / 200]  # [a, b, phi, np.pi * 2 / period]

##Plot the Tracked/target Pixel Data & Add Plot & Data Labels 
plt.plot(xdata, ydata2, "m^", label="pixel 2")
plt.title(title_2)
plt.xlabel(xlabel_2)
plt.ylabel(ylabel_2)
#add data labels (can remove if this clutters the plot)
for value in xdataindex:
    plt.text(xdata[value - 1], ydata2[value - 1], str(ydata2[value - 1]))

##Generate the fitted function
popt, pcov = curve_fit(cosine_function, xdata, ydata2, initialguess2)

##plot the fitted function
fittedxvalues = np.arange(1, xdata[-1], 0.1)  ##make the x values for our fitted function. Can change parameters for more precision
fittedyvalues = cosine_function(fittedxvalues, *popt)  ##astrix allows us to pass both elements in popt into the function

##Display our function on our plot 
function_label2 = print_function(popt)
plt.text(0, max(ydata2) + 20, function_label2) ##change the x & y coordinates of the function's location as needed

##Save and show plot
plt.plot(fittedxvalues, fittedyvalues, "r", label="fitted function")
plt.legend()
plt.savefig(plotfilename_2)
plt.show()
plt.clf()

### 3. Calculate and Print Uncertainty of Curve_fit() Function

This is a simple and very basic block of code which prints out the covariance and the standard error, which are the uncertainty values for the function parameters. I personally did not use this much for the 3 PGMI experiments. 

In [ ]:
#Uncertainty printing & calculations 
print_function(popt)
print(pcov)
stat_errors = np.sqrt(np.diag(pcov)) ##from numpy curve_fit() documentation
print(stat_errors)